# Train YOLOv8 for tic-tac-toe on ROCm

Prerequisites:
- ROCm-ready PyTorch environment
- This repo as current working directory
- Optional: real/images and real/labels with manual annotations (use --real-data real)

Adjust the commands to your environment as needed.


In [ ]:
# Install ROCm PyTorch + dependencies (adjust for your ROCm version)
!pip install --extra-index-url https://download.pytorch.org/whl/rocm6.0 torch torchvision torchaudio
!pip install ultralytics opencv-python

In [ ]:
# Check ROCm device
import torch
print('torch version:', torch.__version__)
print('cuda available (ROCm):', torch.cuda.is_available())
if torch.cuda.is_available():
    print('device count:', torch.cuda.device_count())
    print('device 0 name:', torch.cuda.get_device_name(0))

In [ ]:
# Train and run inference
# Note: use 'cuda' for ROCm-enabled PyTorch (maps to HIP)
!python main.py \
  --image real/images/top-camera05.jpg \
  --output overlay_rocm.jpg \
  --force-regen --force-train \
  --epochs 20 --conf 0.5 \
  --device cuda \
  --grayscale --clahe --contrast-alpha 0.8 --contrast-beta -2 \
  --max-det 30 --train-nms-time 5 \
  --num-train 600 --num-val 120 \
  --real-data real \
  --preprocess-train \
  --save-preprocessed tmp/pre_rocm.jpg

[info] Generating synthetic dataset at data/synth_grid
[info] copied 58 real images into train set
[info] preprocessed 1974 images in data/synth_grid/images/train (including augmented alpha variants)
[info] preprocessed 360 images in data/synth_grid/images/val (including augmented alpha variants)
[info] Training lightweight YOLO model to detect grid: models/cell_grid.pt
[W1206 21:45:35.749490675 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1206 21:45:35.752265737 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1206 21:45:35.753807383 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1206 21:45:35.756284711 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1206 21:45:35.757388290 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1206 21:45:35.757586773 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1206 21:45:35.757986396 N

In [ ]:
# Show inference result
from PIL import Image
Image.open('overlay_rocm.jpg')

In [ ]:

# Verify all real images and save predictions to tmp/preds
# Also warn if no IoU>0.3 match per class vs label
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from main import apply_preprocess

weights = Path('models/cell_grid.pt')
model = YOLO(str(weights))
img_dir = Path('real/images')
lbl_dir = Path('real/labels')
out_dir = Path('tmp/preds')
out_dir.mkdir(parents=True, exist_ok=True)

def yolo_to_xyxy(label_path, shape):
    h, w = shape[:2]
    boxes = []
    for line in Path(label_path).read_text().splitlines():
        if not line.strip():
            continue
        cls, xc, yc, bw, bh = map(float, line.split())
        x0 = (xc - bw / 2) * w
        y0 = (yc - bh / 2) * h
        x1 = (xc + bw / 2) * w
        y1 = (yc + bh / 2) * h
        boxes.append((int(cls), np.array([x0, y0, x1, y1], dtype=np.float32)))
    return boxes

def iou(box1, box2):
    xa = max(box1[0], box2[0]); ya = max(box1[1], box2[1])
    xb = min(box1[2], box2[2]); yb = min(box1[3], box2[3])
    inter = max(0, xb - xa) * max(0, yb - ya)
    area1 = max(0, box1[2]-box1[0]) * max(0, box1[3]-box1[1])
    area2 = max(0, box2[2]-box2[0]) * max(0, box2[3]-box2[1])
    union = area1 + area2 - inter + 1e-6
    return inter / union

for img_path in img_dir.glob('**/*'):
    if img_path.suffix.lower() not in ('.jpg', '.jpeg', '.png'):
        continue
    try:
        img = cv2.imread(str(img_path))
        img = apply_preprocess(img, grayscale=True, clahe=True, contrast_alpha=0.8, contrast_beta=-2)
        res = model.predict(source=img, conf=0.5, imgsz=640, device='cuda', verbose=False)
        cv2.imwrite(str(out_dir / img_path.name), res[0].plot())
        lbl_path = lbl_dir / img_path.relative_to(img_dir)
        lbl_path = lbl_path.with_suffix('.txt')
        if lbl_path.exists():
            gt = yolo_to_xyxy(lbl_path, img.shape)
            preds = [(int(b.cls.item()), b.xyxy[0].cpu().numpy()) for b in res[0].boxes]
            for cls, gt_box in gt:
                matches = [p for p in preds if p[0] == cls and iou(gt_box, p[1]) > 0.3]
                if not matches:
                    print(f"[warn] mismatch {img_path.name}: missing cls {cls}")
    except Exception as e:
        print(f"[error] {img_path}: {e}")

print('done')
